In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
with np.load('/Users/KP/Desktop/UfT/ECE521_Winter_2018/Assignment 3/Assignment 3/notMNIST.npz') as data :
    Data, Target = data ["images"], data["labels"]
    np.random.seed(521)
    randIndx = np.arange(len(Data))
    np.random.shuffle(randIndx)
    Data = Data[randIndx]/255.
    Target = Target[randIndx]
    trainData, trainTarget = Data[:15000], Target[:15000]
    validData, validTarget = Data[15000:16000], Target[15000:16000]
    testData, testTarget = Data[16000:], Target[16000:]

In [ ]:
trainZeros=np.zeros((15000, 10))
trainZeros[np.arange(15000),trainTarget]=1
trainTarget = trainZeros
validZeros=np.zeros((1000, 10))
validZeros[np.arange(1000),validTarget]=1
validTarget = validZeros
testZeros=np.zeros((2724, 10))
testZeros[np.arange(2724),testTarget]=1
testTarget = testZeros

In [ ]:
learning_rate = 0.005
n_epochs = 20000
batch_size = 500
n_dim = 28*28

In [ ]:
def grab_batches(trainData, trainTarget, batch_size):
    batch_indices = np.random.permutation(range(15000)).reshape(-1, batch_size)
    X_batches = trainData.reshape(-1, n_dim)[batch_indices]
    y_batches = trainTarget[batch_indices]
    batches = zip(X_batches, y_batches)
    return batches

In [ ]:
X = tf.placeholder(tf.float32,[None,n_dim])
Y = tf.placeholder(tf.float32,[None,10])

In [ ]:
def hidden_layer(X, hidden_units):
    x_dimension = X.shape[1].value
    initializer = tf.contrib.layers.xavier_initializer(uniform=False)
    hidden_weights = tf.Variable(initializer([x_dimension, hidden_units]), name='weights')
    hidden_biases = tf.Variable(tf.zeros(hidden_units), name='biases')
    
    return tf.add(tf.matmul(X, hidden_weights), hidden_biases)

In [ ]:
y_ = hidden_layer(X,1000)
y_ = hidden_layer(tf.nn.relu(y_),10)

loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=Y, logits=y_))
#regularizer = tf.nn.l2_loss(W)
#loss = tf.reduce_mean(loss + weight_decay * regularizer)

prediction = tf.cast(tf.round(tf.argmax(y_,1)), tf.int8)
equality = tf.equal(prediction, tf.cast(tf.argmax(Y,1), tf.int8))
accuracy = tf.reduce_mean(tf.cast(equality, tf.float32))

training_step = tf.train.AdamOptimizer(learning_rate).minimize(loss)
init = tf.global_variables_initializer()

In [ ]:
weight_decays = [0.0]
valid_accuracies = []
train_accuracies = []
test_accuracies = []

In [ ]:
with tf.Session() as sess:
    for wd in weight_decays:
        sess.run(init)
        print("Weight Decay: {} \n".format(wd))
        for epoch in range(1,n_epochs+1):
            batches = grab_batches(trainData, trainTarget, batch_size)
            for X_batch, y_batch in batches:
                sess.run(training_step, feed_dict={X: X_batch, Y: y_batch})
            feed_dict ={X: trainData.reshape(-1,n_dim), Y: trainTarget}
            train_loss, train_accuracy = sess.run([loss, accuracy], feed_dict)
            print("Epoch: {}, Loss: {}, Accuracy: {}".format(epoch, train_loss, train_accuracy))
        train_accuracies.append(train_accuracy)
        valid_accuracy = sess.run(accuracy, feed_dict = {X: validData.reshape(-1,n_dim), Y: validTarget})
        valid_accuracies.append(valid_accuracy)
        test_accuracy = sess.run(accuracy, feed_dict = {X: testData.reshape(-1,n_dim), Y: testTarget})
        test_accuracies.append(test_accuracy)